# ShOESS experiment sensitivity

This widget draws the sensitivity of the ShOESS (Short baseline Oscillations at the ESS) experiment, assuming a cylinder-shaped detector with a radius of 2 m, filled with mineral oil. Cross-section values are taken from the [OscSNS proposal](https://arxiv.org/abs/1307.7097).

In [23]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import poisson, norm, rv_histogram
import ipywidgets as widgets

plt.rcParams.update({
    "font.family": "serif"
})

In [24]:
antinumu_spectrum = np.loadtxt("data/antinumu_spectrum.csv", delimiter=",")
nue_spectrum = np.loadtxt("data/nue_spectrum.csv", delimiter=",")
miniboone = np.loadtxt("data/numu_disapp.csv", delimiter=",")
microboone = np.loadtxt("data/numu_app.csv", delimiter=",")
gallex = np.loadtxt("data/gallex.csv", delimiter=",")
gallex2 = np.loadtxt("data/gallex2.csv", delimiter=",")
reno = np.loadtxt("data/nue_disapp.csv", delimiter=",")
lsnd1 = np.loadtxt("data/lsnd1.csv", delimiter=",")
lsnd2 = np.loadtxt("data/lsnd2.csv", delimiter=",")
lsnd3 = np.loadtxt("data/lsnd3.csv", delimiter=",")

e_numu = 29.8

es = np.linspace(0.5,53,53*2+1)

bins = np.linspace(0,53,53*2+1)
bin_edges = bins[::2]
bin_centers = bins[1::2]

def f_antinumu(x):
    return np.interp(x,antinumu_spectrum[:,0], antinumu_spectrum[:,1])

def f_nue(x):
    return np.interp(x,nue_spectrum[:,0], nue_spectrum[:,1])

n = f_antinumu(bin_centers)
antinumu_pdf = rv_histogram((n, bin_edges))

n = f_nue(bin_centers)
nue_pdf = rv_histogram((n, bin_edges))

In [25]:
def appearance_prob(sin2theta2, deltam2, e, l):
    return sin2theta2*(np.sin(1.27*deltam2*l/e)**2)

def disappearance_prob(sin2theta2, deltam2, e, l):
    return 1 - sin2theta2*np.sin(1.27*deltam2*l/e)**2

In [33]:
style = {"description_width": "initial"}
layout = widgets.Layout(width="500px")
detector_efficiency = widgets.FloatSlider(
    description="Detector efficiency",
    style=style,
    layout=layout,
    min=0,
    max=1,
    value=0.5,
    continuous_update=False
)

beam_on_efficiency = widgets.FloatSlider(
    description="Beam-on efficiency",
    style=style,
    layout=layout,
    min=0,
    max=1,
    value=0.5,
    continuous_update=False
)

years = widgets.IntSlider(
    description="Years",
    min=1,
    value=3,
    max=5,
    layout=layout,
    style=style,
    continuous_update=False
)

ess_flux = widgets.IntSlider(
    description="ESS flux (x SNS flux)",
    style=style,
    layout=layout,
    min=1,
    value=10,
    max=20,
    continuous_update=False
)

distance = widgets.IntSlider(
    description="Distance from target [m]",
    style=style,
    layout=layout,
    min=1,
    max=100,
    value=60,
    continuous_update=False
)

length = widgets.FloatSlider(
    description="Detector length [m]",
    style=style,
    layout=layout,
    min=1,
    value=4,
    max=20,
    continuous_update=False
)


In [34]:
SNS_DISTANCE = 60
OSC_SNS_LENGTH = 41.61


def draw_poisson_contour(x, y, ax, sig, bkg, levels, colors):
    h = ax.contour(x,
                   y, 
                   poisson.sf(k=sig+bkg, mu=bkg), 
                   levels,
                   colors=colors)
    
    return h


def gui(detector_efficiency, beam_on_efficiency, years, distance, length, ess_flux):
    flux_norm = detector_efficiency*beam_on_efficiency*length/OSC_SNS_LENGTH*ess_flux*years*(SNS_DISTANCE**2/distance**2)

    deltam2s = np.logspace(-2,2,80)
    sin2theta2s = np.logspace(-4,0,80)
    ls = np.linspace(distance-length/2,distance+length/2,10)
    s, d, e, l = np.meshgrid(sin2theta2s, deltam2s, es, ls, indexing='ij')

    app_antinue_probs = np.mean(np.average(appearance_prob(s, d, e, l), axis=2, weights=antinumu_pdf.pdf(es)), axis=2)
    app_nue_probs = np.mean(appearance_prob(s, d, e_numu, l), axis=(2,3))
    disapp_numu_probs = np.mean(disappearance_prob(s, d, e_numu, l), axis=(2,3))
    disapp_nue_probs = np.mean(np.average(disappearance_prob(s, d, e, l), axis=2, weights=nue_pdf.pdf(es)), axis=2)

    fig, ax = plt.subplots(1,3, layout='constrained', figsize=(12,4))

    sig_antinue_app = 207350*0.89*app_antinue_probs*flux_norm
    bkg_antinue_app = 42/0.5/0.5*beam_on_efficiency*detector_efficiency

    s_2d = np.mean(s,axis=(2,3))
    d_2d = np.mean(d,axis=(2,3))

    levels = [1-norm.cdf(5), 1-norm.cdf(3), 0.05]
    colors = ['gold', 'lightcoral', 'mediumseagreen']

    draw_poisson_contour(s_2d, d_2d, ax[0], sig_antinue_app, bkg_antinue_app, levels, colors)

    l1, = ax[0].plot(np.nan, np.nan, color='gold', label=r'ShOESS $5\sigma$ CL')
    l2, = ax[0].plot(np.nan, np.nan, color='lightcoral', label=r'ShOESS $3\sigma$ CL')
    l3, = ax[0].plot(np.nan, np.nan, color='mediumseagreen', label=r'ShOESS 95\% CL')

    ax[0].set_xlabel(r"$\sin^22 \theta_{\mu e}$")
    ax[0].set_ylabel(r"$\Delta m^2_{14}$ [eV$^2$]")
    ax[0].set_ylim(1e-2, 1e2)
    ax[0].set_title(r"$\bar{\nu}_{\mu}\rightarrow\bar{\nu}_e$ (%i year%s)" % (years, 's' if years > 1 else ''))

    fig.legend((l1, l2, l3), ('ShOESS $5\sigma$ CL', 'ShOESS $3\sigma$ CL', 'ShOESS 95% CL'), 
               loc='outside upper center', ncol=3, frameon=False)

    m, = ax[0].plot(microboone[:,0][np.argsort(microboone[:,1])], microboone[:,1][np.argsort(microboone[:,1])], ls='--')
    l1, = ax[0].fill(lsnd1[:,0], lsnd1[:,1], c='lightblue')
    l2, = ax[0].fill(lsnd2[:,0], lsnd2[:,1], c='lightblue')
    l3, = ax[0].fill(lsnd3[:,0], lsnd3[:,1], c='lightblue')

    ax[0].legend([m, l1], [r"MicroBooNE 95% CL ($\nu_e$ app.)", "LSND 99% CL (allowed)"])

    bkg_dis_numu = 745/0.5/0.5*flux_norm*disapp_numu_probs
    sig_dis_numu = 745/0.5/0.5*flux_norm*(1-disapp_numu_probs)

    draw_poisson_contour(s_2d, d_2d, ax[1], sig_dis_numu, bkg_dis_numu, levels, colors)

    ax[1].set_xlabel(r"$\sin^22 \theta_{\mu \mu}$")
    ax[1].set_xlabel(r"$\sin^22 \theta_{\mu \mu}$")
    ax[1].set_ylim(1e-2, 50)
    ax[1].set_xlim(1e-2, 1)

    ax[1].set_title(r"$\nu_{\mu}\rightarrow \nu_s$ (%i year%s)" % (years, 's' if years > 1 else ''))

    ax[1].plot(miniboone[:,0][np.argsort(miniboone[:,1])], miniboone[:,1][np.argsort(miniboone[:,1])], ls='--', label='MiniBooNE+SciBooNE 90% CL')
    ax[1].legend()

    bkg_dis_nue = 2353/0.5/0.5*flux_norm*disapp_nue_probs
    sig_dis_nue = 2353/0.5/0.5*flux_norm*(1-disapp_nue_probs)

    draw_poisson_contour(s_2d, d_2d, ax[2], sig_dis_nue, bkg_dis_nue, levels, colors)

    ax[2].set_xlabel(r"$\sin^22 \theta_{ee}$")
    ax[2].set_ylim(1e-1, 40)
    ax[2].set_xlim(6e-4, 1)

    ax[2].set_title(r"$\nu_e\rightarrow \nu_s$ (%i year%s)" % (years, 's' if years > 1 else ''))

    ax[2].plot(reno[:,0][np.argsort(reno[:,1])], reno[:,1][np.argsort(reno[:,1])], ls='--', label='RENO+NEOS 95% CL')

    ax[2].fill(gallex[:,0], gallex[:,1], label='GALLEX+SAGE+BEST\n95% CL (allowed)', c='lightblue')
    ax[2].fill(gallex2[:,0], gallex2[:,1], c='lightblue')
    ax[2].legend()

    for i in range(3):
        ax[i].set_xscale("log")
        ax[i].set_yscale("log")
        ax[i].grid()

    # fig.savefig("ShOESS.pdf", transparent=True)


In [35]:
out = widgets.interactive_output(
    gui,
    {
        "detector_efficiency": detector_efficiency,
        "beam_on_efficiency": beam_on_efficiency,
        "years": years,
        "distance": distance,
        "length": length,
        "ess_flux": ess_flux
    },
)
ui = widgets.VBox(
    [
        detector_efficiency,
        beam_on_efficiency,
        years,
        distance,
        length,
        ess_flux
    ]
)

# out_loader = widgets.Output()
# with out_loader:
display(ui, out)
    # out.clear_output()

Output()